In [1]:
import os
def find_repo_root(startpath):
    current_path = os.path.abspath(startpath) # Path started on
    while True:
        if os.path.isdir(os.path.join(current_path, '.git')) or os.path.isfile(os.path.join(current_path, 'README.md')): # If on git path, return it
            return current_path
        
        parent_path = os.path.dirname(current_path)

        if parent_path == current_path: # If current path is parent path, stop 
            break
        current_path = parent_path # Set current path to parent path, to check if git path again
    return None # If an issue arises 
root = find_repo_root(os.getcwd())
        

In [ ]:
import numpy as np
import pandas as pd
import csv 

filename = f"{root}/Code/Alien Stuff/Alien Tropes - CodeTest.tsv"
def genre_shift(file_1, file_2, genre):
    df_matrix_t, all_tropes_list = get_matrix()
    make_movie_tropes_df(file_1, df_matrix_t, genre)

In [ ]:
def get_matrix(): # basically copied from timeline_with_genre copy 3

    all_tropes = []
    matrix_maker = []
    genres = []
    with open(f"{root}/Data/liteweight/2020_genre_counts_by_trope.csv", 'r') as file:
        reader = csv.reader(file)
        row1 = next(reader)
        for i in range(2,29):
                genres.append(row1[i])
        for line in reader:
            #line_list = line.split(",")
            all_tropes.append(line[1])
            genre_counts = []
            for i in range(2,29):
                genre_counts.append(int(line[i]))
            normalization = sum(genre_counts)
            try:
                genre_percents = [genre_count / normalization for genre_count in genre_counts]
                matrix_maker.append(genre_percents)
            except ZeroDivisionError:
                matrix_maker.append(genre_counts)
        
    bad_matrix = np.array(matrix_maker)
    matrix = bad_matrix.transpose() 
    df_matrix = pd.DataFrame(matrix)
    df_matrix.columns = all_tropes
    df_matrix.index = genres
    df_matrix.reset_index(names="genre")

    df_matrix_t = df_matrix.transpose()
    df_matrix_t.reset_index(names='Trope', inplace=True)
    df_matrix_t.sort_values(by='Action')
    return df_matrix_t, all_tropes

def make_movie_tropes_df(file_1, df_matrix_t, genre):
    movie_tropes = pd.read_csv(file_1, sep = '\t')
    movie_tropes['Trope'] = movie_tropes['Trope'].apply(lambda x: x.replace(" ", ""))
    movie_tropes = movie_tropes.merge(right=df_matrix_t, on='Trope', how='left',)
    movie_tropes['Duration'] = movie_tropes['End Time'] - movie_tropes['Start Time']
    movie_tropes['Duration'].value_counts()
    active_tropes = movie_tropes[(movie_tropes['Background?'] == 'No') & (movie_tropes['Duration'] < 500)].copy()
    active_tropes['seconds'] = active_tropes.apply(lambda row: set(range(row['Start Time'], row['End Time'] + 1)), axis=1)

    # Get the set of all unique seconds by combining all the 'seconds' sets
    all_seconds = set().union(*active_tropes['seconds'])
    active_tropes.sort_values(by=genre, inplace=True, ascending=False)
    # print(active_tropes['Duration'].value_counts()
    print(active_tropes['seconds'].head())

    #Just use the build_snapshots function from time_lag_genrefication_FJK.ipynb 

In [ ]:
def H(x,t):
    if x < t:
        return 0
    elif x >= t:
        return 1
def vec_filler(x, file_):
    movie_vec = [0] * len(all_tropes)
    with open(file_) as file:
        tsv_reader = csv.reader(file, delimiter='\t')
        next(tsv_reader)
        for line in tsv_reader:
            trope = line[0].replace(" ", "")
            if trope in all_tropes:
                vec_index = all_tropes.index(trope)
                if line[1] == "Yes" or line[2] == "Yes" and line[5] != "Yes":
                    movie_vec[vec_index] -= H(x, float(line[6]))
                elif line[5] == "Yes":
                    pass
                else:
                    movie_vec[vec_index] += H(x, float(line[6]))
            elif trope not in all_tropes:
                pass ### Need to find a way to keep track of this
    normalizer = sum(movie_vec)
    movie_vec = [element / normalizer for element in movie_vec]
    movie_vec = np.array(movie_vec)
    return movie_vec

def vec_filler2(x, file_, all_tropes_list):
    movie_vec = [0] * len(all_tropes_list) # Make vector for length of each trope
    df = pd.read_csv(file_, delimiter = '\t') # Turn csv into dataframe
    df['Trope'] = df['Trope'].apply(lambda x: x.replace(" ", ''))
    print(df.head())
    for trope in df["Trope"]:
        if trope in all_tropes_list:
            vec_index = all_tropes_list.index(trope)
            tropedex = df['Trope'] == trope
            if df['Inverted?/Defied?'][tropedex] == "Yes" or df['Averted?/Subverted?'][tropedex] == "Yes" and df['Background'][tropedex]!= "Yes":
                movie_vec[vec_index] -= H(x, float(df['Start Time'][tropedex]))
            elif df['Setups'][tropedex] == "Yes":
                pass
            else:
                movie_vec[vec_index] += H(x, float(df['Start Time'][tropedex]))
        elif trope not in all_tropes:
            pass ### Need to find a way to keep track of this
    normalizer = sum(movie_vec)
    movie_vec = [element / normalizer for element in movie_vec]
    movie_vec = np.array(movie_vec)
    return movie_vec

    


In [55]:
genre_shift(filename, "hi", 'Horror')

195                                               {5220}
169    {725, 726, 727, 728, 729, 730, 731, 732, 733, ...
100                                               {5649}
96                                                {2314}
229                                               {3410}
Name: seconds, dtype: object


In [ ]:
# Open csvs
# Make trope matrix df
# Add tau to the end time of everything EXCEPT background tropes, and cut off everything at the end of the time series
# Define function that adds a column to the df made out of step functions
# Use matrix to determine values of each genre at each second, by multiplying the step functions vector by the matrix
# Add these values to dataframe along with the contributing tropes during that time step
# Convert seconds to percentage, and remove unnecessary columns

# THEN worry about cleaning up code